Algorithm source: https://agtb.wordpress.com/2009/07/13/auction-algorithm-for-bipartite-matching/ 

In [11]:
# Graph
# Copyright 2013, Michael H. Goldwasser
#
# Developed for use with the book:
#
#    Data Structures and Algorithms in Python
#    Michael T. Goodrich, Roberto Tamassia, and Michael H. Goldwasser
#    John Wiley & Sons, 2013
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

class Graph:
  """Representation of a simple graph using an adjacency map."""

  #------------------------- nested Vertex class -------------------------
  class Vertex:
    """Lightweight vertex structure for a graph."""
    __slots__ = '_element'
  
    def __init__(self, x):
      """Do not call constructor directly. Use Graph's insert_vertex(x)."""
      self._element = x
  
    def element(self):
      """Return element associated with this vertex."""
      return self._element
    
    def set_element(self,_x):
      """change what is stored at the destinatin of the pointer _element"""
      self._element = _x
  
    def __hash__(self):         # will allow vertex to be a map/set key
      return hash(id(self))

    def __str__(self):
      return str(self._element)

    def __lt__(self, other):  # maybe I should take the node with greater height
      return self
    
  #------------------------- nested Edge class -------------------------
  class Edge:
    """Lightweight edge structure for a graph."""
    __slots__ = '_origin', '_destination', '_element'
  
    def __init__(self, u, v, x):
      """Do not call constructor directly. Use Graph's insert_edge(u,v,x)."""
      self._origin = u
      self._destination = v
      self._element = x
  
    def endpoints(self):
      """Return (u,v) tuple for vertices u and v."""
      return (self._origin, self._destination)
  
    def opposite(self, v):
      """Return the vertex that is opposite v on this edge."""
      if not isinstance(v, Graph.Vertex):
        raise TypeError('v must be a Vertex')
      return self._destination if v is self._origin else self._origin
      raise ValueError('v not incident to edge')
  
    def element(self):
      """Return element associated with this edge."""
      return self._element
    
    def set_element(self, _x):
        self._element = _x
        """change what is stored at the destinatin of the pointer _element"""
      
  
    def __hash__(self):         # will allow edge to be a map/set key
      return hash( (self._origin, self._destination) )

    def __str__(self):
      return '({0},{1},{2})'.format(self._origin,self._destination,self._element)
    
  #------------------------- Graph methods -------------------------
  def __init__(self, directed=False):
    """Create an empty graph (undirected, by default).

    Graph is directed if optional paramter is set to True.
    """
    self._outgoing = {}
    # only create second map for directed graph; use alias for undirected
    self._incoming = {} if directed else self._outgoing

  def _validate_vertex(self, v):
    """Verify that v is a Vertex of this graph."""
    if not isinstance(v, self.Vertex):
      raise TypeError('Vertex expected')
    if v not in self._outgoing:
      raise ValueError('Vertex does not belong to this graph.')
    
  def is_directed(self):
    """Return True if this is a directed graph; False if undirected.

    Property is based on the original declaration of the graph, not its contents.
    """
    return self._incoming is not self._outgoing # directed if maps are distinct

  def vertex_count(self):
    """Return the number of vertices in the graph."""
    return len(self._outgoing)

  def vertices(self):
    """Return an iteration of all vertices of the graph."""
    return self._outgoing.keys()

  def edge_count(self):
    """Return the number of edges in the graph."""
    total = sum(len(self._outgoing[v]) for v in self._outgoing)
    # for undirected graphs, make sure not to double-count edges
    return total if self.is_directed() else total // 2

  def edges(self):
    """Return a set of all edges of the graph."""
    result = set()       # avoid double-reporting edges of undirected graph
    for secondary_map in self._outgoing.values():
      result.update(secondary_map.values())    # add edges to resulting set
    return result

  def get_edge(self, u, v):
    """Return the edge from u to v, or None if not adjacent."""
    self._validate_vertex(u)
    self._validate_vertex(v)
    return self._outgoing[u].get(v)        # returns None if v not adjacent

  def degree(self, v, outgoing=True):   
    """Return number of (outgoing) edges incident to vertex v in the graph.

    If graph is directed, optional parameter used to count incoming edges.
    """
    self._validate_vertex(v)
    adj = self._outgoing if outgoing else self._incoming
    return len(adj[v])

  def incident_edges(self, v, outgoing=True):   
    """Return all (outgoing) edges incident to vertex v in the graph.

    If graph is directed, optional parameter used to request incoming edges.
    """
    self._validate_vertex(v)
    adj = self._outgoing if outgoing else self._incoming
    for edge in adj[v].values():
      yield edge

  def insert_vertex(self, x=None):
    """Insert and return a new Vertex with element x."""
    v = self.Vertex(x)
    self._outgoing[v] = {}
    if self.is_directed():
      self._incoming[v] = {}        # need distinct map for incoming edges
    return v
      
  def insert_edge(self, u, v, x=None):
    """Insert and return a new Edge from u to v with auxiliary element x.

    Raise a ValueError if u and v are not vertices of the graph.
    Raise a ValueError if u and v are already adjacent.
    """
    if self.get_edge(u, v) is not None:      # includes error checking
      raise ValueError('u and v are already adjacent')
    e = self.Edge(u, v, x)
    self._outgoing[u][v] = e
    self._incoming[v][u] = e

In [35]:
from collections import deque #https://docs.python.org/2/tutorial/datastructures.html, 
                             #https://docs.python.org/2/library/collections.html#collections.deque


# Read the graph from a file
file = open("assigm4","r") # ****   CHANGE INPUT GRAPH  ****
for i in range(5):
    file.readline()
sourceline = file.readline()
sinkline = file.readline()
sourcelinelist = sourceline.split()
sinklinelist = sinkline.split()
n_bidders = n_sources = int(sourcelinelist[3])
n_goods = n_sinks = int(sinklinelist[3])
print("n_sources, n_sinks:", n_sources, n_sinks)
for i in range(15):
    file.readline()
infoline = file.readline()
infolinelist = infoline.split()
n_nodes = int(infolinelist[2])
n_arcs = int(infolinelist[3])
print("n_nodes, n_arcs:", n_nodes,n_arcs)

G = Graph(True) # a directed graph
nodeList = [] # list of pointers to the Vertex objects stored in the Graph G
nodeList.append(None) # since input lables start from 1, align the off by one counting to match with the Python index

for i in range(1 ,n_nodes + 1):
    nodeList.append(G.insert_vertex((i,0, None)))      # Vertex _element = (label, p, owner)  *** Key!  

biddersSet = set() # set nodes that are sources (bidders)
goodsSet = set() # set nodes that are sinks (goods)

# Read in sources
for i in range(n_sources):
    line = file.readline()
    linelist = line.split()
    source = int(linelist[1])
    biddersSet.add(nodeList[source])

for i in range(n_bidders+1, n_bidders + n_goods+1):  # index through the nodes that are goods
    goodsSet.add(nodeList[i])

# print("initial goodsSet")
# for node in goodsSet:
#     print(node.element())
    
# read in arcs
for i in range(n_arcs):
    linelist = file.readline().split()
    tail = int(linelist[1])
    head = int(linelist[2])
    weight = int(linelist[3])
    G.insert_edge(nodeList[tail], nodeList[head], weight) # Edge _element = weight
file.close()

queue = deque() #usage: Q.append(node), node = Q.popleft()
for i in range(1,n_bidders+1):
    queue.append(nodeList[i])
delta  = 1 / (n_goods + 1)

# print("BiddersSet")
# for node in biddersSet:
#     print(node.element())

while True:
    try:
#         *****  To debug, uncomment these print lines bellow  ******
#         print("Queue")
#         for q in queue:
#             print(q.element())
#         print("Goods")
#         for i in range(n_bidders, n_bidders + n_goods+1):
#             print(nodeList[i].element())
        node = queue.popleft() # IndexError if empty
        outgoing = G.incident_edges(node,True) # the outgoing edges from node
        maxdiff = -1 # w_{ij} - p_j"
        maxbid = None
        for edge in outgoing:  # "find j that maximizes w_{ij} - p_j"
            weight = edge.element()
            pj = edge.opposite(node).element()[1]
            if weight - pj > maxdiff:
                maxdiff = weight - pj
                maxbid = edge.opposite(node)
        if maxdiff >= 0:
            if maxbid.element()[2] is not None:
                queue.append(nodeList[maxbid.element()[2]])
            maxbid.set_element((maxbid.element()[0], maxbid.element()[1] + delta, node.element()[0]))       
    except IndexError as e: # catch when trying to pop from an empty queue which is an IndexError
        break

print("Solution")
for i in range(1 + n_bidders, n_nodes+1):
    node = nodeList[i]
    print(node.element()[2], "takes", node.element()[0])

n_sources, n_sinks: 6 6
n_nodes, n_arcs: 12 14
Solution
3 takes 7
6 takes 8
4 takes 9
1 takes 10
5 takes 11
2 takes 12
